In [2]:
# ====================================================
# 模型训练
# ====================================================

import sys
import os
import math
import time
import random
import shutil
from pathlib import Path
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import cv2
from PIL import Image
import natsort  #排序
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, Adamax
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from albumentations import (Compose, Normalize, Resize, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate,RandomGridShuffle, Transpose,Rotate)
from albumentations.pytorch import ToTensorV2
import timm
import warnings 
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# ====================================================
# 目录设置
# ====================================================

import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_PATH = './images_output_cnn/'
data = pd.read_csv('./train.csv')

In [3]:
# ====================================================
# 全局参数
# ====================================================

class CFG:
    print_freq=100
    num_workers = 12 #进程数,win系统必须为设置为0
    model_name = 'mobilenetv3_large_100'
    size = 512  #图像大小512*512
    epochs = 30  #训练次数
    factor = 0.4 #学习率每次降低多少
    patience = 3 #容忍网路的性能不提升的次数，高于这个次数就降低学习率
    eps = 1e-6 #lr的最小衰减
    lr = 1e-3
    min_lr = 1e-6
    batch_size = 64
    weight_decay = 1e-6
    gradient_accumulation_steps = 1#累计梯度来解决本地显存不足问题
    max_grad_norm = 10 #梯度的最大范数
    seed = 666 #全局种子
    target_size = 7  #n种类型进行划分
    target_col = 'label'
    n_fold = 5
    criterion = 'LabelSmoothingLoss'#标签平滑
    criterion_params = {'CrossEntropyLoss': {'weight':None,'size_average':None,
                                             'ignore_index':-100,'reduce':None,
                                             'reduction':'mean'},
                        'LabelSmoothingLoss': {'classes':7, 'smoothing':0.05, 'dim':-1},
                        'FocalCosineLoss': {'alpha':1, 'gamma':2 , 'xent':0.1}}
    #trn_fold = [1,2,3,4,5]
    
    
#全局种子
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)#神经网络中，参数
    torch.cuda.manual_seed(seed)#GPU设置随机种子
    torch.backends.cudnn.deterministic = True #cuda的随机数种子,默认算法

seed_torch(seed=CFG.seed)

In [4]:
# ====================================================
# 标签平滑
# ====================================================
class LabelSmoothingLoss(nn.Module): 
    def __init__(self, classes=7, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.classes = classes 
        self.dim = dim 
    def forward(self, input, target): 
        pred = input.log_softmax(dim=self.dim) 
        with torch.no_grad():
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.classes - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [5]:
criterion=LabelSmoothingLoss(**CFG.criterion_params[CFG.criterion]).to(device)#标签平滑

In [6]:
class AverageMeter(object):
    """#计算并存储平均值和当前值"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
        
def timeSince(since, percent):#时间间隔
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s#剩余时间
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))
def asMinutes(s):
    m = math.floor(s / 60)  #分,返回数字的下舍整数
    s -= m * 60             #已过s秒
    return '%dm %ds' % (m, s)

def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [7]:
# ====================================================
# 数据集
# ====================================================

class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        return image, label

In [8]:
###########################################################
#图像增强
##########################################################
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            Transpose(p=0.5),    #转置,将图像行和列互换
            HorizontalFlip(p=0.5),#垂直翻转
            VerticalFlip(p=0.5),#水平翻转
            RandomGridShuffle(grid=(3,3), always_apply=False, p=0.5),#网格洗牌
            Rotate(limit=90, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
            #随机旋转图片
            Resize(CFG.size, CFG.size),
            #归一化,ImageNet的均值和标准差。
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
       return Compose([
           Resize(CFG.size, CFG.size),
           Normalize(
               mean=[0.485, 0.456, 0.406],
               std=[0.229, 0.224, 0.225],
           ),
           ToTensorV2(),
       ])

In [9]:
# ====================================================
# 模型初始化
# ====================================================

class load_init_model(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)#加载模型
        net = Net()
        self.model.classifier=net

    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
###############################
#修改最后一层
################################
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #开始构建线性层
        self.fc1=nn.Linear(1280,400)
        self.fc2=nn.Linear(400,100)
        self.fc3=nn.Linear(100,7)
    def forward(self,x):
        x=x.view(-1,1280)#通过view函数将张量x变成 一维向量
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return  x

In [11]:
##################################################
#网络训练
##################################################
def train_fn(train_loader, model, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    start = time.time()
    global_step = 0
    
    model.train()
    for step, (images, labels) in enumerate(train_loader):
        images = images.to(device)  #数据传到相应设备计算
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(images)
        
        loss = criterion(input=y_preds, target=labels)#标签平滑        
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        else:
            loss.backward()#反向传播计算
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)#计算全局梯度范数
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        end = time.time()
        batch_time.update(time.time() - end)
        
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   ))
    return losses.avg

In [12]:
##################################################
#valid层
##################################################

def valid_fn(valid_loader, model, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(input=y_preds, target=labels)#标签平滑
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.softmax(1).to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        end = time.time()
        batch_time.update(time.time() - end)
        
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [13]:
######################################################
#训练主循环
######################################################
def train_loop(data, fold):
    trnin_index = data[data['fold'] != fold].index
    valid_index = data[data['fold'] == fold].index
    
    train_folds = data.loc[trnin_index].reset_index(drop=True)
    valid_folds = data.loc[valid_index].reset_index(drop=True)
    
    train_dataset = TrainDataset(train_folds, transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, transform=get_transforms(data='valid'))
    
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, 
                              shuffle=True, num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, 
                              shuffle=False, num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    model = load_init_model(CFG.model_name, pretrained=True)
    model.to(device)
    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience,
                                  verbose=True, eps=CFG.eps)
  

    
    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        start_time = time.time()
        avg_loss = train_fn(train_loader, model, optimizer, epoch, scheduler, device)
        
        avg_val_loss, preds = valid_fn(valid_loader, model, device)
        valid_labels = valid_folds[CFG.target_col].values
        scheduler.step(avg_val_loss)
        scheduler.step(avg_val_loss)
        score = get_score(valid_labels, preds.argmax(1))
        print('accuracy_score:',score)
        
        
        if score > best_score:
            best_score = score
            torch.save(model, OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
            print('save the best model')
            print('save the best model')
            print('best_score:',best_score)
            preds = preds
            
    valid_folds[[str(c) for c in range(7)]] = preds
    valid_folds['preds'] = preds.argmax(1)

    return valid_folds
    

In [14]:
def get_result(result_df):
    preds = result_df['preds'].values
    labels = result_df[CFG.target_col].values
    score = get_score(labels, preds)

In [15]:
def main():
    oof_df = pd.DataFrame()
    for fold in range(CFG.n_fold):
        
        print("======= {0} fold=========".format(fold+1))
        _oof_df = train_loop(data, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        get_result(_oof_df)
    get_result(oof_df)
    oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [16]:
if __name__ == '__main__':
    main()

======= 1 fold=========
Epoch: [1][0/33] Data 0.000 (0.000) Elapsed 0m 14s (remain 7m 35s) Loss: 1.9468(1.9468) Grad: 0.3025  
Epoch: [1][32/33] Data 0.000 (0.000) Elapsed 0m 23s (remain 0m 0s) Loss: 1.3134(1.3298) Grad: 2.0030  
EVAL: [0/9] Data 2.955 (2.955) Elapsed 0m 3s (remain 0m 24s) Loss: 1.0600(1.0600) 
EVAL: [8/9] Data 0.000 (0.329) Elapsed 0m 4s (remain 0m 0s) Loss: 1.5913(1.2408) 
accuracy_score: 0.5942549371633752
save the best model
save the best model
best_score: 0.5942549371633752
Epoch: [2][0/33] Data 0.000 (0.000) Elapsed 0m 7s (remain 3m 58s) Loss: 0.9777(0.9777) Grad: 1.0425  
Epoch: [2][32/33] Data 0.000 (0.000) Elapsed 0m 17s (remain 0m 0s) Loss: 1.0702(1.0297) Grad: 1.7883  
EVAL: [0/9] Data 2.747 (2.747) Elapsed 0m 2s (remain 0m 22s) Loss: 0.8580(0.8580) 
EVAL: [8/9] Data 0.000 (0.318) Elapsed 0m 3s (remain 0m 0s) Loss: 1.1833(1.0344) 
accuracy_score: 0.6642728904847397
save the best model
save the best model
best_score: 0.6642728904847397
Epoch: [3][0/33] Data 0

In [5]:
from sklearn.metrics import classification_report
df=pd.DataFrame(pd.read_csv('oof_df.csv',encoding='GBK'))
print(classification_report(df['label'], df['preds']))

              precision    recall  f1-score   support

           0       0.66      0.83      0.74       151
           1       0.49      0.70      0.58       122
           2       0.61      0.45      0.52        66
           3       0.34      0.12      0.18        92
           4       0.81      0.88      0.84        33
           5       0.69      0.54      0.60        54
           6       0.97      0.82      0.89        39

    accuracy                           0.62       557
   macro avg       0.65      0.62      0.62       557
weighted avg       0.60      0.62      0.59       557



In [ ]:
##################################################################
#一下内容是对所有的子图预测概率相加,选取概率最大的作为分类的选项
#################################################################

In [6]:
df = pd.read_csv('./oof_df.csv')
result_df = pd.read_csv('./rock_label.csv',encoding='GBK')
df.head()

,image_id,label,fold,0,1,2,3,4,5,6,preds
0,1_2.bmp,1,0,0.004318,0.959787,0.004109,0.005047,0.011146,0.006536,0.009059,1
1,1_5.bmp,1,0,0.005965,0.952053,0.005230,0.008619,0.012052,0.006345,0.009736,1
2,1_8.bmp,1,0,0.004616,0.968674,0.003598,0.006369,0.007163,0.004348,0.005232,1
3,2_1.bmp,6,0,0.005773,0.005810,0.007537,0.006670,0.006135,0.005930,0.962144,6
4,2_6.bmp,6,0,0.009843,0.009663,0.010891,0.010237,0.010922,0.008309,0.940135,6


In [ ]:
#增加列
result_df['0'] =  0.0
result_df['1'] =  0.0
result_df['2'] =  0.0
result_df['3'] =  0.0
result_df['4'] =  0.0
result_df['5'] =  0.0
result_df['6'] =  0.0
result_df['preds'] =  None
#result_df['preds'] =  None
result_df

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['label'], df['preds']))

In [ ]:

for loc_index in range (result_df.shape[0]):
    #print('star')
    #print(loc_index)
    a = result_df[loc_index:loc_index+1]
    main_name = int(a['样本编号'].values)
    #print(main_name)
    for loc_idx in range(df.shape[0]):    
        b = df[loc_idx:loc_idx+1]#取出行
        image_name = str(b['image_id'].values)
        filename, name_suffix = os.path.splitext(image_name)
        name = filename.split('_')[0]  # '-'前面的文件名
        m_name = int( name.split("'")[1])
        #print(m_name)
        if m_name == main_name:
            a = result_df[loc_index:loc_index+1]
            #print(m_name)
            a['0'] = float(b['0'].values) + float(a['0'].values)
            a['1'] = float(b['1'].values) + float(a['1'].values)
            a['2'] = float(b['2'].values) + float(a['2'].values)
            a['3'] = float(b['3'].values) + float(a['3'].values)
            a['4'] = float(b['4'].values) + float(a['4'].values)
            a['5'] = float(b['5'].values) + float(a['5'].values)
            a['6'] = float(b['6'].values) + float(a['6'].values)
            result_df[loc_index:loc_index+1] = a
            #print(a)
        


In [ ]:
result_df

In [ ]:
for loc_index in range (result_df.shape[0]):
    a = result_df[loc_index:loc_index+1]#取出列
    list1 = [float(a['0'].values),
         float(a['1'].values),
         float(a['2'].values),
         float(a['3'].values),
         float(a['4'].values),
         float(a['5'].values),
         float(a['6'].values),]
    a['preds'] = list1.index(max(list1))
    result_df[loc_index:loc_index+1] = a
    
    

In [ ]:
from sklearn.preprocessing import LabelEncoder
#标签编码
le=LabelEncoder()
le.fit(result_df['样本类别'].values.tolist())
le.classes_
result_df['样本类别']=le.transform(result_df['样本类别'].values.tolist())
result_df.rename(columns={'样本类别':'label'},inplace=True)
result_df

In [ ]:
result_df.to_csv("./result.csv", encoding="GBK",index=False)#save file

In [ ]:
result_df = pd.read_csv('./result.csv',encoding='GBK')
print(classification_report(result_df['label'], result_df['preds']))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(result_df['label'], result_df['preds']))